# 'monGPT

This program builds plausible gen 8 ou teams using minGPT

In [3]:
import os

import torch
from showdown_dataset import StartingPokemonDataset, PokemonDataset, RandomizeTeamWrapper, InputTeamWrapper, GPTTeamWrapper
from team_generator import predict_starting_pokemon, decode_pokemon_team, tokenize_pokemon_team

from mongpt.model import GPT
from mongpt.trainer import Trainer

In [4]:
save_dir = 'models'
os.makedirs(save_dir, exist_ok=True)

# load the dataset
dataset = StartingPokemonDataset('dataset/gen9ou/replays', num_starting_pokemon=1)

# randomly split the dataset into an 80/20 train/test split
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [int(len(dataset) * 0.8), len(dataset) - int(len(dataset) * 0.8)])

train_dataset = RandomizeTeamWrapper(train_dataset, num_starting_pokemon=1)
train_dataset = GPTTeamWrapper(train_dataset, num_starting_pokemon=1)

test_dataset = GPTTeamWrapper(test_dataset, num_starting_pokemon=1)

Loading replays: 100%|██████████| 1250/1250 [00:00<00:00, 6443.35it/s]


In [5]:
# create the minGPT model

model_config = GPT.get_default_config()
model_config.model_type = 'gpt-mini'
model_config.vocab_size = train_dataset.get_vocab_size()
model_config.block_size = train_dataset.get_block_size()
model = GPT(model_config)

number of parameters: 2.72M


In [6]:
# create the trainer
train_config = Trainer.get_default_config()
train_config.learning_rate = 5e-4 # the model we're using is so small that we can go a bit faster
train_config.max_iters = 2000
trainer = Trainer(train_config, model, train_dataset)

running on device cuda


In [7]:
# create an evaluation function

def eval_model(model, dataset):
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=False)
    num_correct = 0
    for x, y in dataloader:
        x = x.to(trainer.device)
        y = y.to(trainer.device)
        truth = y[:, -1:]
        preds = model.generate(x, 1, no_duplicates=False, select_from_team=True)[:, -1:]
        num_correct += (preds == truth).sum()
    return num_correct / len(dataset)

In [8]:
def batch_end_callback(trainer):
    if trainer.iter_num % 100 == 0:
        print(f"iter_dt {trainer.iter_dt * 1000:.2f}ms; iter {trainer.iter_num}: train loss {trainer.loss.item():.5f}")


def epoch_end_callback(trainer): # this requires our custom `monGPT
    train_accuracy = eval_model(trainer.model, train_dataset)
    test_accuracy = eval_model(trainer.model, test_dataset)
    print(f"epoch {trainer.epoch_num}: train accuracy {train_accuracy:.2f}; test accuracy {test_accuracy:.2f}")

trainer.set_callback('on_batch_end', batch_end_callback)
trainer.set_callback('on_epoch_end', epoch_end_callback)

trainer.run()

iter_dt 0.00ms; iter 0: train loss 5.53282
iter_dt 12.00ms; iter 100: train loss 3.10730
epoch 1: train accuracy 0.43; test accuracy 0.33
iter_dt 13.00ms; iter 200: train loss 2.64355
iter_dt 14.00ms; iter 300: train loss 2.05980
epoch 2: train accuracy 0.51; test accuracy 0.36
iter_dt 13.00ms; iter 400: train loss 1.87896
iter_dt 13.00ms; iter 500: train loss 1.67000
epoch 3: train accuracy 0.59; test accuracy 0.35
iter_dt 12.00ms; iter 600: train loss 1.43283
epoch 4: train accuracy 0.64; test accuracy 0.35
iter_dt 14.00ms; iter 700: train loss 1.37142
iter_dt 12.00ms; iter 800: train loss 1.08136
epoch 5: train accuracy 0.69; test accuracy 0.40
iter_dt 13.00ms; iter 900: train loss 1.15502
iter_dt 13.00ms; iter 1000: train loss 0.93650
epoch 6: train accuracy 0.72; test accuracy 0.34
iter_dt 15.00ms; iter 1100: train loss 0.89358
epoch 7: train accuracy 0.77; test accuracy 0.35
iter_dt 13.00ms; iter 1200: train loss 0.66273
iter_dt 13.00ms; iter 1300: train loss 0.82238
epoch 8: tra

In [9]:
# save the model
torch.save(model.state_dict(), os.path.join(save_dir, 'monGPT_starter.pt'))

# Evaluation

Now we can calculate the accuracy of the model on the dataset

In [10]:
# create the dataloader
dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

In [11]:
# x, y = next(iter(dataloader))
#
# x = x.to(trainer.device)
#
# model.generate(x, 1, no_duplicates=False, select_from_team=True)

In [12]:
num_correct = 0
for batch in dataloader:
    x, y = batch
    x = x.to(trainer.device)
    y = y.to(trainer.device)
    #print('x,y = ', x, y)
    truth = y[:, -1:]
    preds = model.generate(x, 1, no_duplicates=False, select_from_team=True)[:, -1:]
    #print('Truth = ', truth)
    #print('Preds = ', preds)
    num_correct += (preds == truth).sum()

print(f"Accuracy: {num_correct / len(test_dataset):.2f}")

Accuracy: 0.37


# Inference

Now, lets see how well the model can predict the starting pokemon of a team

In [13]:
x, _ = test_dataset[0]
team1 = x[:6]
team2 = x[6:12]
team1 = decode_pokemon_team(team1)
team2 = decode_pokemon_team(team2)
print(team1, team2)


['Greninja', 'Tyranitar', 'Haxorus', 'Cinderace', 'Zoroark-Hisui', 'Samurott'] ['Dragonite', 'Ceruledge', 'Abomasnow', 'Gyarados', 'Garchomp', 'Mimikyu']


In [14]:
print(predict_starting_pokemon(team1, team2, 1, model))

['Cinderace']
